In [ ]:
!pip install plotly==4.8
!pip install pandas_ta

In [ ]:
import plotly.express as px
import pandas_ta as ta
import pandas as pd
import numpy as np
import random

#graphs all columns for x, uses df.index as y
def graph_stock(df, title):
  #time_column is a string

  pd.options.plotting.backend = "plotly"
  #fig = px.line(df, x='time', y=['price', 'price_2'])
  fig = px.line(df, x=df.index, y=df.columns, title=title)
  fig.show()

def graph_OCHL(df_OCHL, title):
  #fig_1 = px.line(df, x=df.index, y=df.columns, title=title)
  fig_2 = go.Figure(data=go.Ohlc(x=df_OCHL.index,
                      open=df_OCHL['open'],
                      high=df_OCHL['high'],
                      low=df_OCHL['low'],
                      close=df_OCHL['close']))
  fig_2.update(layout_xaxis_rangeslider_visible=False)
  fig_2.show()

##BTC data

In [ ]:
import pandas as pd
import pandas_ta as ta

# Load data
df = pd.read_csv('/content/BTC-USD.csv')
df

# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df.pop('Date')), inplace=True)
df = df[0:360] #deleting NaN values in this hideous .csv

#requires Close (surprsingly is not case sensitive)
#if I have other columns (lets say Close and Close_2), the other columns are ignored
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)
df

In [ ]:
# List of all indicators
df.ta.indicators()

In [ ]:
#exploring functions: ta.kind
ta.rsi()

In [ ]:
#create a new strategy
MyStrategy = ta.Strategy(
    name="DCSMA10",
    ta=[
        {"kind": "ohlc4"},
        {"kind": "sma", "length": 10},
        {"kind": "donchian", "lower_length": 10, "upper_length": 15},
        #{"kind": "ema", "close": "OHLC4", "length": 10, "suffix": "OHLC4"},
    ]
)

df.ta.strategy(MyStrategy)
df

In [ ]:
df.ta.strategy("Momentum")
df

In [ ]:
prova = pd.DataFrame([x for x in range(10)])
prova.index = df.index[0:10]
prova.columns = ['Close']
prova

# (1) Create the Strategy
MyStrategy = ta.Strategy(
    name="DCSMA10",
    ta=[
        {"kind": "sma", "length": 4},
        #{"kind": "ema", "close": "OHLC4", "length": 10, "suffix": "OHLC4"},
    ]
)

# (2) Run the Strategy
prova.ta.strategy(MyStrategy)
prova

##custom rolling function

In [ ]:
df = pd.DataFrame([x*100 for x in range(10)], dtype='float64')
df

#custom function to create a rolling indicator
def custom_function(*args):
  #equivalent of sum()
  total = 0
  for _ in list(args[0]): #the only way to access all the values in the current window
    total += _
  return total

#create new 
df[1] = df.rolling(4).apply(custom_function)

##random stock

In [ ]:
from scipy.stats import skewnorm

def simulate_stock(initial_price, drift, volatility, trend, days):

  def create_pdf(sd, mean, alfa):
    #invertire il segno di alfa
    x = skewnorm.rvs(alfa, size=1000000) 
    def calc(k, sd, mean):
      return (k*sd)+mean
    x = calc(x, sd, mean) #standard distribution

    #graph pdf
    #pd.DataFrame(x).hist(bins=100)

    #pick one random number from the distribution
    #formally I would use cdf, but I just have to pick randomly from the 1000000 samples
    #np.random.choice(x)
    return x

  def create_empty_df(days):
    #creare un empty DataFrame con le date
    empty = pd.DatetimeIndex(
        pd.date_range("2020-01-01", periods=days, freq="D")
    )
    empty = pd.DataFrame(empty)
    #si tagliano ore, minuti, secondi
    empty

    #si tagliano ore, minuti, secondi
    empty.index = [str(x)[0:empty.shape[0]] for x in list(empty.pop(0))]
    empty

    #final dataset con values
    stock = pd.DataFrame([x for x in range(0, empty.shape[0])])
    stock.index = empty.index
    return stock

  #skeleton
  stock = create_empty_df(days)

  #initial price
  stock[0][0] = initial_price

  #create entire stock DataFrame
  x = create_pdf(volatility, drift, trend)
  for _ in range(1, stock.shape[0]):
    stock.iloc[_] = stock.iloc[_-1]*(1+np.random.choice(x))
  
  stock.index = pd.DatetimeIndex(stock.index)
  return stock

In [ ]:
#normal distribution for difference between low and high
def create_pdf(sd, mean, alfa):
  #invertire il segno di alfa
  x = skewnorm.rvs(alfa, size=1000000) 
  def calc(k, sd, mean):
    return (k*sd)+mean
  x = calc(x, sd, mean) #standard distribution

  #graph pdf
  #pd.DataFrame(x).hist(bins=100)

  #pick one random number from the distribution
  #formally I would use cdf, but I just have to pick randomly from the 1000000 samples
  #np.random.choice(x)
  return x

ran = create_pdf(0.1, 0.2, 0)
np.random.choice(ran)

##version_1

In [ ]:
df['close'] = np.random.rand() #tutti uguali
df['close'] = np.random.rand(df.shape[0]) #tutti diversi

In [ ]:
.apply(lambda x : x+200*(abs(np.random.choice(ran))))

In [ ]:
np.random.choice(create_pdf(0.3, 0, 0), 5)

In [ ]:
#market standard deviation = 0.05
df = simulate_stock(1000, 0, 0.05, 0, 1000)
df.columns = ['low']

df['high'] = df['low']+100*abs(np.random.choice(create_pdf(0.05, 0, 0), df.shape[0]))

df['range'] = df['high'] - df['low']

df['open'] = df['low']+df['range']*np.random.rand(df.shape[0])
df['close'] = df['low']+df['range']*np.random.rand(df.shape[0])

df.pop('range')
#df['min'] = df[['rand_1','rand_2']].min(axis=1)
#df['max'] = df[['rand_1','rand_2']].max(axis=1)
df['range'] = abs(df['low']-df['high'])

df = df[['open', 'high', 'low', 'close', 'range']]

import plotly.graph_objects as go
import pandas as pd
print(df['range'].mean())
fig = go.Figure(data=go.Ohlc(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close']))
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

##version_2

In [ ]:
def OCHL(group_values):
  min_ = min(group_values)
  max_ = max(group_values)
  range = max_ - min_
  open = min_+range*random.random()
  close = min_+range*random.random()
  return min_, max_, open, close

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df = simulate_stock(1000, 0, 0.01, 0, 8760)

df_ = list()
#df.groupby(np.arange(len(df))//24).apply(OCHL) non funziona
#sarebbe il modo corretto, ma devo creare un nuovo df da 0
for a, b in df.groupby(np.arange(len(df))//24):
  group_values = np.array(b.values).flatten()
  low, high, open, close = OCHL(group_values)
  df_.append([low, high, open, close])

#
df_OCHL = pd.DataFrame(df_, index=pd.Series(pd.date_range("2020-01-01", periods=365, freq="D")), columns=['low', 'high', 'open', 'close'])

#graph
graph_stock(df, "")
fig = go.Figure(data=go.Ohlc(x=df_OCHL.index,
                    open=df_OCHL['open'],
                    high=df_OCHL['high'],
                    low=df_OCHL['low'],
                    close=df_OCHL['close']))
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
df_OCHL

In [ ]:
#groupby day
#df.index da errore
#dobbaimo convertirlo a df.DatetimeIndex
#quindi df.index = pd.DatetimeIndex(df.index)
df.groupby(pd.Grouper(freq='D')).mean()

In [ ]:
#groupby interval of 24 rows
df.groupby(np.arange(len(df))//24).mean()

In [ ]:
#groupby month
df.groupby(pd.Grouper(freq='M')).mean()

##pandas_ta

In [ ]:
df.ta.indicators()

In [ ]:
#little handmade stock
def little():
  df = pd.DataFrame([x*100 for x in range(3)]+[300-x*50 for x in range(4)]+[40+x*130 for x in range(4)]+[390-x*25 for x in range(3)], dtype='float64')
  df.columns = ['close']
  return df

In [ ]:
#random stock
def big():
  df = simulate_stock(1000, 0.000001, 0.01, 0, 8760)

  df_ = list()
  #df.groupby(np.arange(len(df))//24).apply(OCHL) non funziona
  #sarebbe il modo corretto, ma devo creare un nuovo df da 0
  for a, b in df.groupby(np.arange(len(df))//24):
    group_values = np.array(b.values).flatten()
    low, high, open, close = OCHL(group_values)
    df_.append([low, high, open, close])

  #
  df_OCHL = pd.DataFrame(df_, index=pd.Series(pd.date_range("2020-01-01", periods=365, freq="D")), columns=['low', 'high', 'open', 'close'])
  return df, df_OCHL

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df, df_OCHL = big()

#graph
graph_stock(df, "")
graph_OCHL(df_OCHL, "")

df = df_OCHL.copy()
df

In [ ]:
#generate stock
df, df_OCHL = big()
df = df_OCHL.copy()

#apply indicator
MyStrategy = ta.Strategy(
    name="DCSMA10",
    ta=[
        {"kind": "sma", "length" : 20},
        {"kind": "ao"},
    ]
)

df.ta.strategy(MyStrategy)
df.index = pd.date_range("2020-01-01", periods=df.shape[0], freq="D")

#in case of visible returns
#df[df.columns[1]] *= 5000
graph_stock(df.dropna(), "all_indicators")

#determine BUY and SELL + graph
import numpy as np
import math

#pick ONE indicator to analyze
df = df[['close', 'SMA_20']]
indicator_name = df.columns[1]
df.columns = ['price', 'indicator']
df

#when price is above SMA, uptrend
#when price is below SMA, downtrend
#df['signal'] = np.where((df['price']-df['indicator'])>0, 'BUY', 'SELL')
graph_stock(df[['price', 'indicator']].dropna(), indicator_name)

#graph OCHL
graph_OCHL(df_OCHL, "")

.